In [17]:
%pylab inline

import networkx as nx
import scipy.sparse
import pandas as pd
from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance

from regain import datasets; reload(datasets)
from regain.datasets import is_pos_def, is_pos_semidef
from regain.plot import plot_graph_with_latent_variables
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_)
from regain.admm import latent_time_graph_lasso_v2_; reload(latent_time_graph_lasso_v2_)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_);
from regain.admm import latent_graph_lasso_; reload(latent_graph_lasso_);
from regain.utils import error_norm

Populating the interactive namespace from numpy and matplotlib


/home/veronica/anaconda3/envs/palladio/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['beta', 'eig']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
%load_ext pymatbridge
#%matlab cd /home/fede/Downloads/lvglasso-pub/
%matlab cd /home/veronica/src/lvglasso-pub/

In [16]:
#data = pd.read_csv("../regain/data/gene_chandrasekaran.csv").values.T

In [44]:
#admat("../regain/data/X_output_chandra.mat")['X1']
# C2 = loadmat("../regain/data/X2_output_chandra.mat")['X2']
# out_B = loadmat("../regain/data/out_B.mat")
SigmaO = loadmat("../regain/data/SigmaO.mat")

In [ ]:
#emp_cov = empirical_covariance(data_list[0], assume_centered=False)

In [2]:
input_dim=100
sparsity = 0.025

p = input_dim + int(round(input_dim+0.05))
po = input_dim
ph = p - input_dim
W = np.zeros((p,p))
picks = np.random.permutation(p*p)
dim = int(round(p*p*sparsity))
picks = picks[1:dim]
W = W.ravel(order='F')
W[picks] = np.random.randn(dim);

W = np.reshape(W, (p,p), order="F")
C = W.T.dot(W)
C[0:po,po:p] = C[0:po,po:p] + 0.5*np.random.randn(po,ph)
C = (C+C.T)/2;

d = diag(C)
np.clip(C-np.diag(d), -1, 1, out=C)
eig, Q = np.linalg.eigh(C)
K = C + max(-1.2 * np.min(eig), 0.001) * np.eye(p)
KO = K[0:po,0:po]
KOH = K[0:po,po:p]
KHO = K[po:p,0:po]
KH = K[po:p,po:p]
assert(is_pos_semidef(KH))
assert(np.linalg.matrix_rank( np.divide(KOH, KH.dot(KHO))) == ph)
KOtilde = KO - np.divide(KOH, KH.dot(KHO))#
eig,Q = np.linalg.eigh(KOtilde)
#print(eig)
#assert(is_pos_def(KOtilde))
N = 5*po
EmpCov = np.linalg.inv(KOtilde)
EmpCov = (EmpCov + EmpCov.T)/2
data = np.random.multivariate_normal(np.zeros(po), EmpCov, size=N)
SigmaO =(1./N)*data.T.dot(data)


/home/veronica/anaconda3/envs/palladio/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.
  if sys.path[0] == '':
/home/veronica/anaconda3/envs/palladio/lib/python2.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: covariance is not positive-semidefinite.


In [42]:
# config
n_samples = 100
input_dim = 100
n_dim_lat = 7
T = 1
data_list = np.array([data])
data_list
# datasets.generate_ma_xue_zou()

# data_list.shape
# data_grid = np.array(data_list).transpose(1,2,0)
data_list, K, K_obs, ells = datasets.generate_dataset(
     mode='fixed', n_samples=n_samples, n_dim_lat=n_dim_lat, n_dim_obs=input_dim, T=T)

data_grid = np.array(data_list).transpose(1,2,0)  # to use it later for grid search

In [43]:
print(ells)

[[ 0.09513259  0.03368829  0.02753087 ...,  0.04339444  0.05060072
   0.02225323]
 [ 0.03368829  0.04608059  0.03688512 ...,  0.01716031  0.04078696
   0.00849314]
 [ 0.02753087  0.03688512  0.03907343 ...,  0.01869296  0.02714915
   0.00062201]
 ..., 
 [ 0.04339444  0.01716031  0.01869296 ...,  0.05283863  0.00598158
   0.00910157]
 [ 0.05060072  0.04078696  0.02714915 ...,  0.00598158  0.06081169
   0.01231768]
 [ 0.02225323  0.00849314  0.00062201 ...,  0.00910157  0.01231768
   0.01278422]]


In [79]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from regain import utils; reload(utils)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_)
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_);
from regain.admm.latent_time_graph_lasso_ import LatentTimeGraphLasso
ltgl = GridSearchCV(LatentTimeGraphLasso(bypass_transpose=False, assume_centered=True),
                    dict(tau=np.logspace(0,1,10),#[0.25, 0.01, 0.1]
                         eta=[0],
                         alpha=np.logspace(0,1,10),#[0.05, 0.1, 1], #
                         beta=[0]),
                    cv=ShuffleSplit(10)).fit(data_grid)


[8]
(1.0, 1.0, 49.823256218511972)
[8]
(1.0, 1.0, 46.404459436698041)
[8]
(1.0, 1.0, 52.695294515796796)
[8]
(1.0, 1.0, 46.413780891290486)
[9]
(1.0, 1.0, 50.819237529341542)
[9]
(1.0, 1.0, 46.428901027296057)
[9]
(1.0, 1.0, 48.476468528441949)
[9]
(1.0, 1.0, 46.367149013940789)
[9]
(1.0, 1.0, 50.858507123492394)
[9]
(1.0, 1.0, 46.458302090184269)
[9]
(1.0, 1.0, 47.590211806402692)
[9]
(1.0, 1.0, 46.345249808141425)
[9]
(1.0, 1.0, 48.741795836442641)
[9]
(1.0, 1.0, 46.708915873225536)
[9]
(1.0, 1.0, 51.427558533032617)
[9]
(1.0, 1.0, 46.29027692779151)
[8]
(1.0, 1.0, 51.081568960344313)
[8]
(1.0, 1.0, 46.332877608531867)
[8]
(1.0, 1.0, 48.572277710758925)
[8]
(1.0, 1.0, 46.627360339607002)
[7]
(1.0, 1.2915496650148839, 49.644377687406859)
[7]
(1.0, 1.2915496650148839, 47.138992672241415)
[7]
(1.0, 1.2915496650148839, 52.579933130157443)
[7]
(1.0, 1.2915496650148839, 47.103174551527545)
[7]
(1.0, 1.2915496650148839, 50.746506914499818)
[7]
(1.0, 1.2915496650148839, 47.143038215103218)
[

[0]
(1.0, 7.7426368268112693, 50.485268004367633)
[0]
(1.0, 7.7426368268112693, 50.122269626741627)
[0]
(1.0, 7.7426368268112693, 51.938816743562413)
[0]
(1.0, 7.7426368268112693, 50.040595672487385)
[0]
(1.0, 7.7426368268112693, 51.366441384496852)
[0]
(1.0, 7.7426368268112693, 50.059552470126057)
[0]
(1.0, 7.7426368268112693, 48.901120983090834)
[0]
(1.0, 7.7426368268112693, 50.196077378516804)
[0]
(1.0, 10.0, 49.854004240997746)
[0]
(1.0, 10.0, 50.164781478868925)
[0]
(1.0, 10.0, 53.235485821283092)
[0]
(1.0, 10.0, 49.97425052584488)
[0]
(1.0, 10.0, 50.916574646414226)
[0]
(1.0, 10.0, 50.094144067352737)
[0]
(1.0, 10.0, 48.424803358050575)
[0]
(1.0, 10.0, 50.221022998671046)
[0]
(1.0, 10.0, 50.665032756815478)
[0]
(1.0, 10.0, 50.100975234889326)
[0]
(1.0, 10.0, 47.314233953344555)
[0]
(1.0, 10.0, 50.283706844171384)
[0]
(1.0, 10.0, 50.485268004367633)
[0]
(1.0, 10.0, 50.122269626741627)
[0]
(1.0, 10.0, 51.938816743562413)
[0]
(1.0, 10.0, 50.040595672487385)
[0]
(1.0, 10.0, 51.366441

[0]
(1.2915496650148839, 3.5938136638046276, 44.794407916238207)
[0]
(1.2915496650148839, 3.5938136638046276, 43.733225317672726)
[0]
(1.2915496650148839, 3.5938136638046276, 42.683328476224041)
[0]
(1.2915496650148839, 3.5938136638046276, 43.865083923950252)
[0]
(1.2915496650148839, 4.6415888336127784, 43.528821338935643)
[0]
(1.2915496650148839, 4.6415888336127784, 43.831008596082178)
[0]
(1.2915496650148839, 4.6415888336127784, 46.434044415288469)
[0]
(1.2915496650148839, 4.6415888336127784, 43.6465407783394)
[0]
(1.2915496650148839, 4.6415888336127784, 44.426534175981431)
[0]
(1.2915496650148839, 4.6415888336127784, 43.764441655938278)
[0]
(1.2915496650148839, 4.6415888336127784, 42.262074794326615)
[0]
(1.2915496650148839, 4.6415888336127784, 43.890106242282499)
[0]
(1.2915496650148839, 4.6415888336127784, 44.19660466519359)
[0]
(1.2915496650148839, 4.6415888336127784, 43.772660774080997)
[0]
(1.2915496650148839, 4.6415888336127784, 41.310432584489774)
[0]
(1.2915496650148839, 4.6

[2]
(1.6681005372000588, 1.6681005372000588, 37.600113209988919)
[2]
(1.6681005372000588, 1.6681005372000588, 37.341759598881559)
[3]
(1.6681005372000588, 1.6681005372000588, 38.763943989341875)
[3]
(1.6681005372000588, 1.6681005372000588, 37.05414372658808)
[3]
(1.6681005372000588, 1.6681005372000588, 38.249997067565374)
[3]
(1.6681005372000588, 1.6681005372000588, 36.976647414572135)
[2]
(1.6681005372000588, 1.6681005372000588, 36.646226766789063)
[2]
(1.6681005372000588, 1.6681005372000588, 37.157171143109899)
[2]
(1.6681005372000588, 2.1544346900318838, 37.367107885691091)
[2]
(1.6681005372000588, 2.1544346900318838, 37.529620509833336)
[1]
(1.6681005372000588, 2.1544346900318838, 39.79407254368089)
[1]
(1.6681005372000588, 2.1544346900318838, 37.270210889769317)
[1]
(1.6681005372000588, 2.1544346900318838, 38.103378188699075)
[1]
(1.6681005372000588, 2.1544346900318838, 37.408150394643528)
[2]
(1.6681005372000588, 2.1544346900318838, 36.260367546026515)
[2]
(1.6681005372000588, 2.

[0]
(1.6681005372000588, 10.0, 37.405746131770258)
[0]
(1.6681005372000588, 10.0, 37.691086600941453)
[0]
(1.6681005372000588, 10.0, 39.863534293220845)
[0]
(1.6681005372000588, 10.0, 37.518093192243484)
[0]
(1.6681005372000588, 10.0, 38.152421582819905)
[0]
(1.6681005372000588, 10.0, 37.630371721361882)
[0]
(1.6681005372000588, 10.0, 36.302275727547119)
[0]
(1.6681005372000588, 10.0, 37.75075789385685)
[0]
(1.6681005372000588, 10.0, 37.946764095814018)
[0]
(1.6681005372000588, 10.0, 37.639558287291415)
[0]
(1.6681005372000588, 10.0, 35.49902621127179)
[0]
(1.6681005372000588, 10.0, 37.807193374692915)
[0]
(1.6681005372000588, 10.0, 37.854249649939646)
[0]
(1.6681005372000588, 10.0, 37.654716225939474)
[0]
(1.6681005372000588, 10.0, 38.927468647770731)
[0]
(1.6681005372000588, 10.0, 37.579223300510741)
[0]
(1.6681005372000588, 10.0, 38.448755827700751)
[0]
(1.6681005372000588, 10.0, 37.603129326392931)
[0]
(1.6681005372000588, 10.0, 36.668595439873613)
[0]
(1.6681005372000588, 10.0, 37

...

[0]
(4.6415888336127784, 10.0, 17.738416466667029)
[0]
(4.6415888336127784, 10.0, 17.912970489925666)
[0]
(4.6415888336127784, 10.0, 18.846738554061002)
[0]
(4.6415888336127784, 10.0, 17.810792430510407)
[0]
(4.6415888336127784, 10.0, 18.056149613104928)
[0]
(4.6415888336127784, 10.0, 17.881070660322695)
[0]
(4.6415888336127784, 10.0, 17.195516475130322)
[0]
(4.6415888336127784, 10.0, 17.957335936121293)
[0]
(4.6415888336127784, 10.0, 17.947817478347709)
[0]
(4.6415888336127784, 10.0, 17.889432163089907)
[0]
(4.6415888336127784, 10.0, 16.835384545441887)
[0]
(4.6415888336127784, 10.0, 17.99046152949958)
[0]
(4.6415888336127784, 10.0, 17.934366735295047)
[0]
(4.6415888336127784, 10.0, 17.893468860067607)
[0]
(4.6415888336127784, 10.0, 18.429871446740851)
[0]
(4.6415888336127784, 10.0, 17.847809740084333)
[0]
(4.6415888336127784, 10.0, 18.169179113771268)
[0]
(4.6415888336127784, 10.0, 17.868784654363395)
[0]
(4.6415888336127784, 10.0, 17.375123848791539)
[0]
(4.6415888336127784, 10.0, 1

[0]
(5.9948425031894086, 3.5938136638046276, 14.656088351247243)
[0]
(5.9948425031894086, 3.5938136638046276, 14.427810805339115)
[0]
(5.9948425031894086, 3.5938136638046276, 14.022050491925954)
[0]
(5.9948425031894086, 3.5938136638046276, 14.488151795445878)
[0]
(5.9948425031894086, 4.6415888336127784, 14.316935958023167)
[0]
(5.9948425031894086, 4.6415888336127784, 14.463373434609336)
[0]
(5.9948425031894086, 4.6415888336127784, 15.203593057431858)
[0]
(5.9948425031894086, 4.6415888336127784, 14.378270970691563)
[0]
(5.9948425031894086, 4.6415888336127784, 14.568420323312715)
[0]
(5.9948425031894086, 4.6415888336127784, 14.437475205890323)
[0]
(5.9948425031894086, 4.6415888336127784, 13.876381979859806)
[0]
(5.9948425031894086, 4.6415888336127784, 14.501839475902687)
[0]
(5.9948425031894086, 4.6415888336127784, 14.479649226955832)
[0]
(5.9948425031894086, 4.6415888336127784, 14.444902704425207)
[0]
(5.9948425031894086, 4.6415888336127784, 13.588529813814066)
[0]
(5.9948425031894086, 

[0]
(7.7426368268112693, 1.6681005372000588, 11.58530528120049)
[0]
(7.7426368268112693, 1.6681005372000588, 11.569909338709362)
[0]
(7.7426368268112693, 1.6681005372000588, 11.902890106462801)
[0]
(7.7426368268112693, 1.6681005372000588, 11.538706684011844)
[0]
(7.7426368268112693, 1.6681005372000588, 11.727985882630797)
[0]
(7.7426368268112693, 1.6681005372000588, 11.554689210775619)
[0]
(7.7426368268112693, 1.6681005372000588, 11.224917152023243)
[0]
(7.7426368268112693, 1.6681005372000588, 11.604152403681887)
[0]
(7.7426368268112693, 2.1544346900318838, 11.462065376282554)
[0]
(7.7426368268112693, 2.1544346900318838, 11.583017412087981)
[0]
(7.7426368268112693, 2.1544346900318838, 12.166703659793281)
[0]
(7.7426368268112693, 2.1544346900318838, 11.513171466511524)
[0]
(7.7426368268112693, 2.1544346900318838, 11.660123067435951)
[0]
(7.7426368268112693, 2.1544346900318838, 11.562226728050019)
[0]
(7.7426368268112693, 2.1544346900318838, 11.107837223114567)
[0]
(7.7426368268112693, 2

[0]
(7.7426368268112693, 10.0, 11.462065376282554)
[0]
(7.7426368268112693, 10.0, 11.583017412087981)
[0]
(7.7426368268112693, 10.0, 12.166703659793281)
[0]
(7.7426368268112693, 10.0, 11.513171466511524)
[0]
(7.7426368268112693, 10.0, 11.660123067435951)
[0]
(7.7426368268112693, 10.0, 11.562226728050019)
[0]
(7.7426368268112693, 10.0, 11.107837223114567)
[0]
(7.7426368268112693, 10.0, 11.615629644068528)
[0]
(7.7426368268112693, 10.0, 11.588189554860385)
[0]
(7.7426368268112693, 10.0, 11.568640086739133)
[0]
(7.7426368268112693, 10.0, 10.879233374126937)
[0]
(7.7426368268112693, 10.0, 11.638278488621429)
[0]
(7.7426368268112693, 10.0, 11.58530528120049)
[0]
(7.7426368268112693, 10.0, 11.569909338709362)
[0]
(7.7426368268112693, 10.0, 11.902890106462801)
[0]
(7.7426368268112693, 10.0, 11.538706684011844)
[0]
(7.7426368268112693, 10.0, 11.727985882630797)
[0]
(7.7426368268112693, 10.0, 11.554689210775619)
[0]
(7.7426368268112693, 10.0, 11.224917152023243)
[0]
(7.7426368268112693, 10.0, 1

[0]
(10.0, 5.9948425031894086, 9.211774139972297)
[0]
(10.0, 5.9948425031894086, 9.2016024994879437)
[0]
(10.0, 5.9948425031894086, 8.6509023838756534)
[0]
(10.0, 5.9948425031894086, 9.258201488062678)
[0]
(10.0, 5.9948425031894086, 9.2112459199414722)
[0]
(10.0, 5.9948425031894086, 9.2022631393377416)
[0]
(10.0, 5.9948425031894086, 9.4629464787038469)
[0]
(10.0, 5.9948425031894086, 9.1768997505789347)
[0]
(10.0, 5.9948425031894086, 9.3219907212303514)
[0]
(10.0, 5.9948425031894086, 9.1903453059508031)
[0]
(10.0, 5.9948425031894086, 8.9249178744256437)
[0]
(10.0, 5.9948425031894086, 9.2304227649550032)
[0]
(10.0, 7.7426368268112693, 9.1141417942272867)
[0]
(10.0, 7.7426368268112693, 9.2127472304718303)
[0]
(10.0, 7.7426368268112693, 9.6711082622128952)
[0]
(10.0, 7.7426368268112693, 9.1561371938033975)
[0]
(10.0, 7.7426368268112693, 9.2695225165662904)
[0]
(10.0, 7.7426368268112693, 9.1961988363066975)
[0]
(10.0, 7.7426368268112693, 8.8315141884619113)
[0]
(10.0, 7.7426368268112693, 9.

In [80]:
emp_list = list(ltgl.best_estimator_.covariance_)
alpha = ltgl.best_params_['alpha']
beta = ltgl.best_params_['beta']
tau = ltgl.best_params_['tau']
print(tau) #0.25
print(alpha)# 0.05

3.5938136638
1.0


In [81]:
results_time = latent_time_graph_lasso_.latent_time_graph_lasso(
        np.array(data_list), alpha=alpha, tau=tau,#0.25,#0.01,
        tol=1e-5, rtol=1e-5, rho=1./data.shape[0],
        verbose=0, max_iter=500)
    

In [82]:
np.linalg.matrix_rank(results_time[1][0])
#print(ltgl.best_score_)#results_time[1][0]
# from palladio import plotting; reload(plotting);
# plotting.score_surfaces_gridsearch(ltgl, indep_vars=['tau','eta'], logspace=['tau','eta'])

40

In [ ]:
R_ar = np.array([R[i*R.shape[1]:(i+1)*R.shape[1]] for i in range(R.shape[0]/R.shape[1])])
L_ar = np.array([L[i*L.shape[1]:(i+1)*L.shape[1]] for i in range(L.shape[0]/L.shape[1])])
K_obs = np.array(K_obs) # K_obs is the matrix of observed samples
K = np.array(K) # K is the real precision matrix
ells = np.array(ells) # the latent matrices

### Error obtained w.r.t. the observed data

In [ ]:

error_norm(ltgl.best_estimator_.precision_, K_obs)

In [ ]:
error_norm(R_ar, K_obs)
#print(ells)

### Error obtained w.r.t. the real data

In [ ]:
print(error_norm(ltgl.best_estimator_.precision_, K))
print(error_norm(ltgl.best_estimator_.latent_, ells))

In [ ]:
print(error_norm(R_ar, K))
print(error_norm(L_ar, ells))

### Error in ranks

In [ ]:
ells_estimated = ltgl.best_estimator_.latent_
print [np.linalg.matrix_rank(ells_estimated[i]) for i in range(ells_estimated.shape[0])]
print [np.linalg.matrix_rank(L_ar[i]) for i in range(ells_estimated.shape[0])]
print np.linalg.matrix_rank(ells)

In [ ]:
#es, Q = np.linalg.eigh(ells_estimated[0])

### Error in structure

In [ ]:
for true, predict in zip(K.copy(), ltgl.best_estimator_.precision_.copy()):
    predict[np.abs(predict)<1e-2] = 0
    print(predict)
    print(true)
    print("________________________________________________________________________________________")
    true[true != 0] = 1
    predict[predict != 0] = 1
    res = true+predict
    #print(res)
    #print(res==1)
    print np.count_nonzero((res==1).astype(int))

In [ ]:
print true

In [ ]:
print K

In [ ]:
print ltgl.best_estimator_.precision_
